# **Lab 3: Iterative Methods**
**Kevin Arnmark**

# **Abstract**

In this report I implement different iterative methods to solve both linear and nonlinear systems. I will be implementing four different methods. The jacobi iteration, Gauss-Seidel iteration, Newton's method and GMRES method.

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [ ]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell.

In [1]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

# **Introduction**

In this report I will construct the following algorithms:


1. **Function:** Jacobi iteration for Ax=b

  **Input:** matrix A, vector b

  **Output:** vector x

  **Test:** convergence of residual || Ax-b ||, || x-y || for manufactured/exact solution y 

2. **Function:** Gauss-Seidel iteration for Ax=b

  **Input:** matrix A, vector b

  **Output:** vector x

  **Test:** convergence of residual || Ax-b ||, || x-y || for manufactured/exact solution y

3. **Function:** Newton's method for scalar nonlinear equation f(x)=0

  **Input:** scalar function f(x)

  **Output:** real number x

  **Test:** convergence of residual |f(x)|, |x-y| for manufactured/exact solution y

**Extra assignment:**

4. **Function:** GMRES method for Ax=b

  **Input:** matrix A, vector b

  **Output:** vector x 

  **Test:** convergence of residual || Ax-b ||, || x-y || for manufactured/exact solution y

# **Method**

Throughout the report I am using the value of $1^{-7}$ for the tolerance of the stopping criterium of the iterative methods. This is to ensure a low enough tolerance to pass the tests. If the tolerance is decreased, the precision is increased.

**Jacobi iteration**

The jacobi iteration is equivalent to a left preconditioned Richardson iteration with $\alpha = 1$. The jacobi iteration is a preconditioned Richardson iteration with $B = D^{-1}$, where $D$ is a matrix with identical diagonal elements as $A$ (Chapter 7.6-7.7). The convergence criteria for this method is $\lvert\lvert I - D^{-1}A \rvert \rvert < 1$. The base algorithm used for the richardson iteration is from Chapter 7.6 Algorithm 7.1.

In [2]:
def pre_richardson_iteration(A, B, b, alpha):
  assert len(A) == len(A[0])

  x = np.zeros(len(A))
  tol = 1e-7

  r = np.array([1, 1, 1]) # placeholder whit a norm larger than tol
  while np.linalg.norm(r) > tol:
    r = b - np.dot(A, x)
    x = x + alpha * np.dot(B, r)

  return x


def jacobi_iteration(A, b):
  id = np.identity(len(A))
  B = np.diag(1. / np.diag(A)) # Taking 1/x for each x in the diagonal of A, the inverse of an diagonal matrix
  assert np.linalg.norm(id - np.matmul(B, A)) < 1 # Checking the convergence criteria

  return pre_richardson_iteration(A, B, b, 1)


**Gauss-Seidel iteration** Solving $Ax=b$

Gauss-Seidel iteration is preconditioned Richardson iteration with $B=L^{-1}$ and $\alpha = 1$ with the convergence criterion $\lvert\lvert I - L^{-1}A \rvert \rvert < 1$. L is the lower triangular matrix constructed from $A$ by zeroing out all the entries above the diagonal (Chapter 7.7 Example 7.9).

In [3]:
def forward_substitution(L, b):
  n = len(b)
  x = np.zeros(n)
  x[0] = b[0] / L[0,0]

  for i in range(1, n):
    s = 0
    for j in range(i):
      s += L[i,j]*x[j]
    x[i] = (b[i] - s) / L[i, i]
  return x

def gauss_seidel_iteration(A, b):
  n = len(A)
  id = np.identity(n)
  L = np.tril(A)
  B = np.zeros((L.shape))
  for i in range(n):
    B[:,i] = forward_substitution(L, id[:,i])

  assert np.linalg.norm(id - np.matmul(B, A)) < 1 # Checking the convergence criteria
  return pre_richardson_iteration(A, B, b, 1)

**Newtons Method**
For solving scalar nonlinear equation $f(x)=0$. 

This algorithm is built upon Algorithm 8.2 in Chapter 8.3. To calculate the derivative of f(x) I used Theorem 8.4 (Mean value theorem) in Chapter 8.2 to get an approximation of the derivative. To get the a good approximation I used the value closest to 0 python allows to calculate a divison as the difference in the input values of the functions.

In [4]:
def newtons_method(fnc, x0):
  x = x0
  tol = 1e-7
  h = 1e-15 # closest number to zero that does not give divison by zero error
  while np.abs(fnc(x)) > tol:
    df = (fnc(x + h) - fnc(x)) / h
    #print(df)
    x -= fnc(x)/df
  return x

**GMRES**

This algorithm is currently not working. I ran out of time to complete it. If you can easily spot the misstakes I would be grateful if you could comment what they are!

This algorithm is based on Algorithm 7.2 in Chapter 7.8. The Arnoldi iteration that is used in the GMRES algorithm is based on Algorithm 7.3. To solve the least square problem I used a sequence of givens rotations to convert the Hessenberg matrix into an upper triangular matrix and then used backwards substitution to solve for $y$. The backward substitution algorithm is based on Algorithm 5.2 in Chapter 5.2.

In [5]:
def arnoldi_iteration(A, b, k):
  n = len(A)
  Q = np.zeros((n, k+1))
  H = np.zeros((k+1, k))
  Q[:,0] = b/np.linalg.norm(b)
  for j in range(k):
    v = np.dot(A, Q[:,j])
    for i in range(j):
      H[i,j] = np.dot(Q[:,i], v)
      v -= H[i,k]*Q[:,i]
    H[j+1,j] = np.linalg.norm(v)
    Q[:,j+1] = v[:] / H[j+1,j]
  return Q, H


def givens_rotation(H, k, i):
  x1 = H[0,0]
  x2 = H[1,0]
  norm = np.linalg.norm(H[:,0])
  c = x1 / norm
  s = -(x2 / norm)
  G = np.identity(k)
  G[i:i+2,i:i+2] = np.array([[c,-s], [s,c]])
  return G

def backward_substitution(U, b):
  n = len(b)
  x = np.zeros((n, ))
  x[n - 1] = b[n - 1] / U[n-1, n-1]
  for i in range(n - 2, -1, -1):
    s = 0
    for j in range(i + 1, n):
      s += U[i, j] * x[j]
    x[i] = (b[i] - s) / U[i, i]
  return x


def least_squares_problem(H, v):
  n = len(H)
  A = np.zeros((2,))
  Gk = np.identity(n)
  
  for k in range(n-1):
    G = givens_rotation(H[k:k+2,k:k+2], n, k)
    Gk = np.matmul(G, Gk)

  H = np.matmul(Gk,H)
  y = backward_substitution(H, np.dot(Gk, v))
  return y


def gmres(A, b):
  n = len(A)
  assert len(A[0]) == n
  assert len(b) == n
  k = 0
  tol = 1e-6
  r = np.array([1,1,1])
  while (np.linalg.norm(r) / np.linalg.norm(b)) > tol:
    k += 1
    e1 = np.zeros((k + 1,))
    e1[0] = 1

    Q, H = arnoldi_iteration(A, b, k)
    y = least_squares_problem(H, np.linalg.norm(b)*e1)
    r = np.linalg.norm(b)*e1 - np.dot(H, y)
  
  x = np.dot(Q[:,:k-1], y)
  return x

# Tests:

Below are some basic tests to ensure the algorithms are working as intended. The tolerance is at 7 decimals.

In [6]:
# Jacobi iteration tests

# Constructing solution
A = np.array([[8,1,0], [0,7,1], [1,0,9]]) # Diagonal dominant matrix
y = [3, -2, 1]
b = np.dot(A, y) 

x = jacobi_iteration(A, b)

np.testing.assert_almost_equal(np.linalg.norm(np.dot(A, x) - b), 0) # Testing the convergence of the residual
np.testing.assert_almost_equal(np.linalg.norm(x - y), 0) # Testing the difference between the exact solution and the jacobi iteration solution

print("Jacobi iteration tests passed") # Prints if nothing fails

# Gauss-Seidel iterations
A = np.array([[2,1,0], [1,3,-1], [1,2,4]])
y = [1, -4, 2]
b = np.dot(A, y) 

x = gauss_seidel_iteration(A, b)

np.testing.assert_almost_equal(np.linalg.norm(np.dot(A, x) - b), 0) # Testing the convergence of the residual
np.testing.assert_almost_equal(np.linalg.norm(x - y), 0) # Testing the difference between the exact solution and the gauss-seidel iteration solution

print("Gauss-Seidel iteration tests passed") # Prints if nothing fails

# Newtons method for scalar nonlinear equation f(x)=0 tests
def fnc1(x):
  return x**2 - 4

def fnc2(x):
  return np.cos(x)

x1 = newtons_method(fnc1, 1)
x2 = newtons_method(fnc2, 1)

np.testing.assert_almost_equal(np.abs(fnc1(x1)), 0) # Testing the convergence of the residual
np.testing.assert_almost_equal(np.abs(fnc2(x2)), 0) # Testing the convergence of the residual

np.testing.assert_almost_equal(np.abs(x1 - 2), 0) # Testing the difference between the exact solution and the newton's method solution
np.testing.assert_almost_equal(np.abs(x2 - np.pi/2), 0) # Testing the difference between the exact solution and the newton's method solution

print("Newton's method tests passed") # Prints if nothing fails

# GMRES tests

Jacobi iteration tests passed
Gauss-Seidel iteration tests passed
Newton's method tests passed


# **Results**

The results are as expected.

# **Discussion**

If I had the time I would have written some more randomized tests and also measured the time of the algorithms to compare their performance.